In [6]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Define the problem
prob = LpProblem("The Diet Problem", LpMinimize)


food_items = ['Whole_Grain_Bread', 'Banana', 'Yogurt', 'Nuts', 'Salmon']
food_vars = LpVariable.dicts("Food", food_items, lowBound=0, cat='Continuous')




costs = {food: 1 for food in food_items} 
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the Diet"

# Nutritional information extracted from the image (per serving)
nutrition_info = {
    'Calories': {'Whole_Grain_Bread': 140, 'Banana': 120, 'Yogurt': 140, 'Nuts': 170, 'Salmon': 520},
    'Protein': {'Whole_Grain_Bread': 4, 'Banana': 1, 'Yogurt': 10, 'Nuts': 5, 'Salmon': 41},
    'Vitamin_D': {'Whole_Grain_Bread': 0, 'Banana': 0, 'Yogurt': 0, 'Nuts': 0, 'Salmon': 22},
    'Calcium': {'Whole_Grain_Bread': 52, 'Banana': 7, 'Yogurt': 130, 'Nuts': 30, 'Salmon': 20},
    'Iron': {'Whole_Grain_Bread': 0, 'Banana': 0, 'Yogurt': 0, 'Nuts': 0.9, 'Salmon': 0.91},
    'Potassium': {'Whole_Grain_Bread': 0, 'Banana': 487, 'Yogurt': 188, 'Nuts': 190, 'Salmon': 800},
    'Sodium': {'Whole_Grain_Bread': 260, 'Banana': 0, 'Yogurt': 80, 'Nuts': 75, 'Salmon': 710}
}


constraints = {
    'Calories': {'min': 2000},
    'Protein': {'min': 50},
    'Vitamin_D': {'min': 20},
    'Calcium': {'min': 1300},
    'Iron': {'min': 18},
    'Potassium': {'min': 4700},
    'Sodium': {'max': 5000},
}


for nutrient, constraint in constraints.items():
    if 'min' in constraint:
        prob += lpSum([nutrition_info[nutrient][i] * food_vars[i] for i in food_items]) >= constraint['min'], f"Min_{nutrient}"
    if 'max' in constraint:
        prob += lpSum([nutrition_info[nutrient][i] * food_vars[i] for i in food_items]) <= constraint['max'], f"Max_{nutrient}"

prob


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


The_Diet_Problem:
MINIMIZE
1*Food_Banana + 1*Food_Nuts + 1*Food_Salmon + 1*Food_Whole_Grain_Bread + 1*Food_Yogurt + 0
SUBJECT TO
Min_Calories: 120 Food_Banana + 170 Food_Nuts + 520 Food_Salmon
 + 140 Food_Whole_Grain_Bread + 140 Food_Yogurt >= 2000

Min_Protein: Food_Banana + 5 Food_Nuts + 41 Food_Salmon
 + 4 Food_Whole_Grain_Bread + 10 Food_Yogurt >= 50

Min_Vitamin_D: 22 Food_Salmon >= 20

Min_Calcium: 7 Food_Banana + 30 Food_Nuts + 20 Food_Salmon
 + 52 Food_Whole_Grain_Bread + 130 Food_Yogurt >= 1300

Min_Iron: 0.9 Food_Nuts + 0.91 Food_Salmon >= 18

Min_Potassium: 487 Food_Banana + 190 Food_Nuts + 800 Food_Salmon
 + 188 Food_Yogurt >= 4700

Max_Sodium: 75 Food_Nuts + 710 Food_Salmon + 260 Food_Whole_Grain_Bread
 + 80 Food_Yogurt <= 5000

VARIABLES
Food_Banana Continuous
Food_Nuts Continuous
Food_Salmon Continuous
Food_Whole_Grain_Bread Continuous
Food_Yogurt Continuous

In [7]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/0feb0a9277704640b4c71e22d2aa9699-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/0feb0a9277704640b4c71e22d2aa9699-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 44 RHS
At line 52 BOUNDS
At line 53 ENDATA
Problem MODEL has 7 rows, 5 columns and 26 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-3) rows, 5 (0) columns and 15 (-11) elements
Perturbing problem by 0.001% of 8.40586 - largest nonzero change 0.00023186723 ( 0.0068959995%) - largest zero change 0
0  Obj 15.22941 Primal inf 12.829472 (3)
2  Obj 25.447847
Optimal - objective value 25.446775
After Postsolve, objec

1

In [8]:
print("Optimal Diet Plan:")
for v in prob.variables():
    print(f"{v.name}: {v.varValue} servings")

Optimal Diet Plan:
Food_Banana: 0.0 servings
Food_Nuts: 19.080808 servings
Food_Salmon: 0.90909091 servings
Food_Whole_Grain_Bread: 0.0 servings
Food_Yogurt: 5.4568765 servings
